In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import seaborn as sns
import  matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Inspecting data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
df.shape

(1460, 81)

In [6]:
null=df.isna().sum()*100/len(df)

In [7]:
null[(null>50)].index

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [8]:
df.drop(null[(null>50)].index,axis=1,inplace=True)

In [9]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [10]:
df.describe(include='object')

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1460,1460,770,1379,1379,1379,1379,1460,1460,1460
unique,5,2,4,4,2,5,3,25,9,8,...,4,7,5,6,3,5,5,3,9,6
top,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal
freq,1151,1454,925,1311,1459,1052,1382,225,1260,1445,...,735,1360,380,870,605,1311,1326,1340,1267,1198


In [11]:
#checking for columns with single value
unique=df.nunique()
unique[unique<=1]

Series([], dtype: int64)

## Imputing null values

In [12]:
#splitting numeric and categorical columns
df_num=df.select_dtypes(include=np.number)
df_cat=df.select_dtypes(exclude=np.number)

In [13]:
df_num.shape,df_cat.shape

((1460, 38), (1460, 39))

In [14]:
num_na=df_num.isna().sum()
num_na[num_na>0]

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [15]:
df[num_na[num_na>0].index].skew()

LotFrontage    2.163569
MasVnrArea     2.669084
GarageYrBlt   -0.649415
dtype: float64

In [16]:
df_num['LotFrontage']=df_num['LotFrontage'].fillna(df_num['LotFrontage'].median())
df_num['MasVnrArea']=df_num['MasVnrArea'].fillna(df_num['MasVnrArea'].median())
df_num['GarageYrBlt']=df_num['GarageYrBlt'].fillna(df_num['GarageYrBlt'].median())

In [17]:
cat_na=df_cat.isna().sum()
cat_na[cat_na>0]

MasVnrType        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
FireplaceQu     690
GarageType       81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64

In [18]:
#Inspecting categorical NaN columns for imputing 
for i in cat_na[cat_na>0].index:
    print(df[i].value_counts())

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64
TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64
TA    1311
Gd      65
Fa      45
Po       2
Name: BsmtCond, dtype: int64
No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64
Unf    1256
Rec      54
LwQ      46
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64
Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64
Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64
Unf    605
RFn    422
Fin    352
Name: GarageFinish, dtype: int64
TA    1311
Fa      48
Gd      14
Ex       3
Po       3
Name: GarageQual, dtype: int64
TA    1326
Fa      35
Gd       9
P

In [19]:
for i in cat_na[cat_na>0].index:
    df_cat[i]=df_cat[i].fillna(method='ffill')
df_cat['FireplaceQu']=df_cat['FireplaceQu'].fillna(df_cat['FireplaceQu'].mode()[0])

In [20]:
df_mod=pd.concat([df_num,df_cat],axis=1)
df_mod.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [21]:
#mean encoding categorical columns
for i in df_cat.columns:
    tab=df_mod.groupby(i)['SalePrice'].mean()
    df_mod[i]=df_mod[i].map(tab)

## Model Building

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn import svm

In [23]:
x=df_mod.drop('SalePrice',axis=1)
y=df_mod[['SalePrice']]

In [24]:
x.shape,y.shape

((1460, 76), (1460, 1))

In [25]:
#feature scaling
ss=StandardScaler()
x_scaled=pd.DataFrame(ss.fit_transform(x),columns=x.columns)

In [26]:
#initializing regression models
clf1=LinearRegression()
clf2=RandomForestRegressor()
clf3=GradientBoostingRegressor()
clf4=KNeighborsRegressor()
clf5=XGBRegressor()
clf6=svm.SVR()

In [27]:
from sklearn.model_selection import cross_val_score
mod=[clf1,clf2,clf3,clf4,clf5,clf6]
mod_name=['LinearR','RF','GradBoost','KNN','XGB','SVM']
for  i ,j in zip(mod,mod_name):
    score=cross_val_score(i,x,y,cv=5,scoring='r2')
    avg_score=np.mean(score)
    var_score=np.std(score)/np.mean(score)*100
    print(j,'Avg Score:',avg_score, ' Variance error: ',var_score)

LinearR Avg Score: 0.8380888366992633  Variance error:  7.218245781664903
RF Avg Score: 0.8654509741010619  Variance error:  3.081828650341778
GradBoost Avg Score: 0.8978775842772461  Variance error:  2.466895459083843
KNN Avg Score: 0.6934756373406717  Variance error:  3.4717256114481536
XGB Avg Score: 0.876306166055288  Variance error:  1.840820028799291
SVM Avg Score: -0.051309702846500646  Variance error:  -36.281010781484355


## feature selection

In [28]:
rfe=RFE(clf5,n_features_to_select=20)#using xgb to select best 20 features

In [29]:
rfe_mod=rfe.fit(x,y)

In [30]:
ranks=pd.Series(rfe_mod.ranking_,index=x.columns)

In [31]:
ranks[ranks==1].index

Index(['OverallQual', 'OverallCond', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'Fireplaces', 'GarageCars',
       'MSZoning', 'Neighborhood', 'Condition1', 'BsmtQual', 'BsmtExposure',
       'CentralAir', 'KitchenQual', 'Functional', 'SaleType', 'SaleCondition'],
      dtype='object')

In [32]:
x_rfe=x[ranks[ranks==1].index]

In [33]:
from sklearn.model_selection import cross_val_score
mod=[clf1,clf2,clf3,clf4,clf5,clf6]
mod_name=['LinearR','RF','GradBoost','KNN','XGB','SVM']
for  i ,j in zip(mod,mod_name):
    score=cross_val_score(i,x_rfe,y,cv=5,scoring='r2')
    avg_score=np.mean(score)
    var_score=np.std(score)/np.mean(score)*100
    print(j,'Avg Score:',avg_score, ' Variance error: ',var_score)

LinearR Avg Score: 0.8311975502402692  Variance error:  6.488373451881673
RF Avg Score: 0.8760731558013258  Variance error:  2.9084536950375086
GradBoost Avg Score: 0.8962623904271503  Variance error:  2.5194869113439893
KNN Avg Score: 0.6986230801149258  Variance error:  3.6254606327888523
XGB Avg Score: 0.8810848796798799  Variance error:  3.027403795954373
SVM Avg Score: -0.05087401129216999  Variance error:  -36.555155477408825


GradBoost with RFE features has achieved almost 90% accuracy which is almost similar to the accuracy achieved without performing feature selection. 